## Imports

In [1]:
import mwapi
import mwparserfromhell as parser
import re
import pandas as pd
import requests
import urllib.parse

## Helper functions and constants

In [ ]:
base_url = "https://wikidata.org/w/api.php"

In [ ]:
# get all the pages that have a certain prefix in a given namespace
def get_pages_with_prefix(prefix, namespace=None):
    params = {
        "action": "query",
        "format": "json",
        "list": "allpages",
        "apnamespace": namespace,
        "apprefix": prefix,
        "aplimit": "max"
    }
    
    try:
        response = requests.get(base_url, params=params)
        response.raise_for_status()  # Check for HTTP errors
        data = response.json()
    except Exception as e:
        print("Error:", e)
        print("Response content:", response.text)
        return []

    # if data is correctly formatted, return
    if 'query' in data and 'allpages' in data['query']:
        pages = data['query']['allpages']
        return pages
    else:
        print("Unexpected response format:", data)
        return []

# helper function for parsing external wikilinks
def parse_external_wikilinks(link):
    (url, text) = link.split(' ', 1)
    text = text[:-1]

    if 'query.wikidata.org/' in url:
        query = urllib.parse.unquote(url.split('query.wikidata.org/')[1].split('#', 1)[1])
        
    elif 'w.wiki/' in url:
        query = ''
        response = requests.get(url[1:], allow_redirects=True) #url[1:] to drop the leading bracket
        if 'query.wikidata.org/' in response.url:
            query = urllib.parse.unquote(response.url.split('query.wikidata.org/')[1].split('#', 1)[1])

    return text, query

# helper functions for labeling QIDs with english labels
def get_qid_titles(qid_set):
    qids = {}
    for qid in qid_set:
        response = requests.get(f'https://www.wikidata.org/w/rest.php/wikibase/v0/entities/items/{qid}?_fields=labels')
        data = response.json()
        try:
            qids[qid] = data['labels']['en']
        except:
            qids[qid] = ''
    return qids

# helper functions for labeling PIDs with english labels
def get_pid_titles(pid_set):
    pids = {}
    for pid in pid_set:
        response = requests.get(f'https://www.wikidata.org/w/rest.php/wikibase/v0/entities/properties/{pid}?_fields=labels')
        data = response.json()
        try:
            pids[pid] = data['labels']['en']
        except:
            pids[pid] = ''
    return pids

# Overview
There are several existing sources of high-quality, human-labeled, and open-source SparQL queries on Wikidata. These queries (sometimes with associated requests, titles, and discussions) can serve as a core of high-quality data for a labeled SparQL dataset.

## Query of the week (QOTW) pages
Archive of all queries spotlighted over 7 years by the "Query of the week" initiative.

Found at https://www.wikidata.org/wiki/Wikidata:SPARQL_query_service/qotw

In [ ]:
def parse_qotw_page(title):
    print(title)
    rows = []
    
    resp = session.get(
        formatversion=2,
        action='query',
        prop='revisions',
        rvslots='*',
        rvprop='content',
        titles=title
    )

    content = resp['query']['pages'][0]['revisions'][0]['slots']['main']['content']
    
    wikitext = parser.parse(content)

    for l in wikitext.filter_external_links():
        if 'query.wikidata.org/' in l or 'w.wiki/' in l:
            try:
                text, query = parse_external_wikilinks(l)
                query_dict = {}
                query_dict['heading'] = text
                query_dict['full_text'] = ''
                query_dict['text_query'] = ''

                # get all QIDs and PIDs
                qid_set = set(re.findall(r"Q\d+", f'{text}\n{query}'))
                pid_set = set(re.findall(r"P\d+", f'{text}\n{query}'))

                query_dict['qids'] = get_qid_titles(qid_set)
                query_dict['pids'] = get_pid_titles(pid_set)
                query_dict['sparql_query'] = query
                query_dict['title'] = title

                rows.append(query_dict)
            except:
                continue
    return rows

In [ ]:
session = mwapi.Session('https://wikidata.org', user_agent='mwapi sparql')
qotw = []

qotw_prefix = "SPARQL query service/qotw"
qotw_pages = get_pages_with_prefix(qotw_prefix, namespace=4)
for page in qotw_pages:
    if re.search(r'Wikidata:SPARQL query service/qotw/\d+', page['title']):
        qotw += parse_qotw_page(page['title'])

pd.DataFrame(qotw).to_pickle('data/high-quality/qotw.pkl')

## Request a query archive pages
Archive of all human query requests made during the ongoing "Request a query" initiative.

Found at https://wikidata.org/wiki/Wikidata:Request_a_query/Archive

In [ ]:
# for a given page title, split by headings and collect:
# - heading
# - full text under the heading
# - initial text query
# - all mentions of wikidata items and properties with english labels
# - sparql query answer
def parse_request_a_query_page(title):
    print(title)
    rows = []
    
    resp = session.get(
        formatversion=2,
        action='query',
        prop='revisions',
        rvslots='*',
        rvprop='content',
        titles=title
    )

    content = resp['query']['pages'][0]['revisions'][0]['slots']['main']['content']
    
    wikitext = parser.parse(content)
    headings = wikitext.filter_headings()

    # split by each heading
    for i in range(len(headings)):
        query_dict = {}
        
        # get heading
        query_dict['heading'] = headings[i]
        
        # get full text
        if i == len(headings) - 1:
            query_dict['full_text'] = wikitext.split(str(headings[i]))[1]
        else:
            query_dict['full_text'] = wikitext.split(str(headings[i]))[1].split(str(headings[i+1]))[0]

        # get initial text query
        text_query = query_dict['full_text'].split("[[User:")[0]
        query_dict['text_query'] = text_query
        
        # get all QIDs and PIDs
        qid_set = set(re.findall(r"Q\d+", query_dict['full_text']))
        pid_set = set(re.findall(r"P\d+", query_dict['full_text']))
        query = ''
        for template in parser.parse(query_dict['full_text']).filter_templates():

            # get QIDs and PIDs that are mentioned in template form
            if '{{Q|' in template:
                qid = template.split('{{Q|')[1][:-2].split('|')[0]
                if 'Q' in qid:
                    qid_set.add(qid)
                else:
                    qid_set.add(f'Q{qid}')

            elif '{{P|' in template:
                pid = template.split('{{P|')[1][:-2].split('|')[0]
                if 'P' in pid:
                    pid_set.add(pid)
                else:
                    pid_set.add(f'P{pid}')

            # get sparql query
            elif '{{SPARQL' in template:
                query = re.split(r"query\s*=\s*", str(template))[1].split("|")[0]
                if query.endswith("\n}}"):
                    query = query[:-3]
                if query.endswith("}}"):
                    query = query[:-2]
                query = query.replace("{{!}}", "|")

        if query == '':
            continue

        query_dict['qids'] = get_qid_titles(qid_set)
        query_dict['pids'] = get_pid_titles(pid_set)
        query_dict['sparql_query'] = query
        query_dict['title'] = title
        
        rows.append(query_dict)
    return rows

In [ ]:
session = mwapi.Session('https://wikidata.org', user_agent='mwapi sparql')
roq = []

archive_prefix = "Request a query"
archive_pages = get_pages_with_prefix(archive_prefix, namespace=4)
for page in archive_pages:
    if re.search(r'Wikidata:Request a query/Archive/\d+/\d+', page['title']):
        roq += parse_request_a_query_page(page['title'])
        
pd.DataFrame(roq).to_pickle('data/high-quality/request-a-query.pkl')

## Example SparQL query pages
There are many example queries available on Wikidata, with headings and labels that are translated into multiple languages.

Found at https://wikidata.org/wiki/Wikidata:SPARQL_query_service/queries/examples and subpages of that page

In [ ]:
def parse_example_page(title):
    print(title)
    rows = []
    
    resp = session.get(
        formatversion=2,
        action='query',
        prop='revisions',
        rvslots='*',
        rvprop='content',
        titles=title
    )

    content = resp['query']['pages'][0]['revisions'][0]['slots']['main']['content']
    
    wikitext = parser.parse(content)
    headings = wikitext.filter_headings()

    # split by each heading
    for i in range(len(headings)):
        query_dict = {}
        
        # get heading
        query_dict['heading'] = headings[i]
        
        # get full text
        if i == len(headings) - 1:
            query_dict['full_text'] = wikitext.split(str(headings[i]))[1]
        else:
            query_dict['full_text'] = wikitext.split(str(headings[i]))[1].split(str(headings[i+1]))[0]

        # get initial text query
        text_query = query_dict['full_text'].split("{{SPARQL")[0]
        query_dict['text_query'] = text_query
        
        # get all QIDs and PIDs
        qid_set = set(re.findall(r"Q\d+", query_dict['full_text']))
        pid_set = set(re.findall(r"P\d+", query_dict['full_text']))
        query = ''
        for template in parser.parse(query_dict['full_text']).filter_templates():

            # get QIDs and PIDs that are mentioned in template form
            if '{{Q|' in template:
                qid = template.split('{{Q|')[1][:-2].split('|')[0]
                if 'Q' in qid:
                    qid_set.add(qid)
                else:
                    qid_set.add(f'Q{qid}')

            elif '{{P|' in template:
                pid = template.split('{{P|')[1][:-2].split('|')[0]
                if 'P' in pid:
                    pid_set.add(pid)
                else:
                    pid_set.add(f'P{pid}')

            # get sparql query
            elif '{{SPARQL' in template:
                query = re.split(r"query\s*=\s*", str(template))[1].split("|")[0]
                if query.endswith("\n}}"):
                    query = query[:-3]
                if query.endswith("}}"):
                    query = query[:-2]
                query = query.replace("{{!}}", "|")

        if query == '':
            continue

        query_dict['qids'] = get_qid_titles(qid_set)
        query_dict['pids'] = get_pid_titles(pid_set)
        query_dict['sparql_query'] = query
        query_dict['title'] = title
        
        rows.append(query_dict)
    return rows

In [ ]:
session = mwapi.Session('https://wikidata.org', user_agent='mwapi sparql')
example = []

example_prefix = "SPARQL query service/queries/examples"
example_pages = get_pages_with_prefix(example_prefix, namespace=4)
for page in example_pages:
    example += parse_example_page(page['title'])

pd.DataFrame(example).to_pickle('data/high-quality/example.pkl')

## Tweets, Facts, and Queries 
WMDE employee Lukas Werkmeister runs the Twitter and Mastodon accounts @WikidataFacts (https://twitter.com/WikidataFacts and https://mastodon.social/@WikidataFacts), which often post a tweet + a query of some kind.

Lukas also has documented many (but not all!) of these queries at https://www.wikidata.org/wiki/User:TweetsFactsAndQueries/Queries and https://www.wikidata.org/wiki/User:TweetsFactsAndQueries/Problems.

In [ ]:
def parse_tfaq_query_page(title):
    print(title)    
    resp = session.get(
        formatversion=2,
        action='query',
        prop='revisions',
        rvslots='*',
        rvprop='content',
        titles=title
    )

    content = resp['query']['pages'][0]['revisions'][0]['slots']['main']['content']
    
    wikitext = parser.parse(content)
    templates = wikitext.filter_templates()
    text = title.split('User:TweetsFactsAndQueries/Queries/')[1]
    query_dict = {}
    try:
        for template in templates:
            if 'query page' in template.name:
                query = re.split(r"query\s*=\s*", str(template['query']))[1]
                query = query.replace("{{!}}", "|")
                if template.has_param('header_wikitext_paragraph'):
                    query_dict['text_query'] = re.split(r"header_wikitext_paragraph\s*=\s*", str(template['header_wikitext_paragraph']))[1]
                if template.has_param('footer_wikitext'):
                    query_dict['text_query'] += "\n" + re.split(r'footer_wikitext\s*=\s*', str(template['footer_wikitext']))[1]
        query_dict['heading'] = text
        query_dict['full_text'] = ''

        # get all QIDs and PIDs
        qid_set = set(re.findall(r"Q\d+", f'{text}\n{query}'))
        pid_set = set(re.findall(r"P\d+", f'{text}\n{query}'))

        query_dict['qids'] = get_qid_titles(qid_set)
        query_dict['pids'] = get_pid_titles(pid_set)
        query_dict['sparql_query'] = query
        query_dict['title'] = title
    except:
        return []

    return [query_dict]

def parse_tfaq_problem_page(title):
    print(title)
    rows = []
    
    resp = session.get(
        formatversion=2,
        action='query',
        prop='revisions',
        rvslots='*',
        rvprop='content',
        titles=title
    )

    content = resp['query']['pages'][0]['revisions'][0]['slots']['main']['content']
    
    wikitext = parser.parse(content)
    
    headings = wikitext.filter_headings()

    # split by each heading
    for i in range(len(headings)):
        if headings[i] == "== Automatic ==" or headings[i] == "== Manual ==":
            continue
        query_dict = {}
        
        # get heading
        query_dict['heading'] = headings[i]
        
        # get full text
        if i == len(headings) - 1:
            query_dict['full_text'] = wikitext.split(str(headings[i]))[1]
        else:
            query_dict['full_text'] = wikitext.split(str(headings[i]))[1].split(str(headings[i+1]))[0]

        # get initial text query
        text_query = query_dict['full_text'].split("]\n")[-1]
        query_dict['text_query'] = text_query
        
        for l in parser.parse(query_dict['full_text']).filter_external_links():
            if 'query.wikidata.org/' in l or 'w.wiki/' in l:
                try:
                    _, query = parse_external_wikilinks(l)

                    # get all QIDs and PIDs
                    qid_set = set(re.findall(r"Q\d+", f'{text_query}\n{query}'))
                    pid_set = set(re.findall(r"P\d+", f'{text_query}\n{query}'))

                    query_dict['qids'] = get_qid_titles(qid_set)
                    query_dict['pids'] = get_pid_titles(pid_set)
                    query_dict['sparql_query'] = query
                    query_dict['title'] = title

                    rows.append(query_dict)
                except:
                    continue
    return rows

In [ ]:
session = mwapi.Session('https://wikidata.org', user_agent='mwapi sparql')
tfaq = []

tfaq_prefix = "TweetsFactsAndQueries/Queries/"
tfaq_pages = get_pages_with_prefix(tfaq_prefix, namespace=2)
for page in tfaq_pages:
    tfaq += parse_tfaq_query_page(page['title'])

tfaq += parse_tfaq_problem_page('User:TweetsFactsAndQueries/Problems')
pd.DataFrame(tfaq).to_pickle('data/high-quality/tfaq.pkl')

### Tweets, facts, and queries Twitter archive

I managed to get the full archive of tweets/retweets directly from Lucas. They're now stored at `data/WikidataFacts-tweets.jsonl`. This section is for parsing those tweets + descriptions + links.

In [ ]:
import json

# helper function for parsing external wikilinks
def parse_tweet_links(url):
    if 'query.wikidata.org/' in url:
        query = urllib.parse.unquote(url.split('query.wikidata.org/')[1].split('#', 1)[1])
        
    elif 'w.wiki/' in url or 'tinyurl.com/' in url:
        query = ''
        response = requests.get(url, allow_redirects=True)
        if 'query.wikidata.org/' in response.url:
            query = urllib.parse.unquote(response.url.split('query.wikidata.org/')[1].split('#', 1)[1])

    return query

def parse_tweet(t):
    rows = []
    for u in t['tweet']['entities']['urls']:
        if 'query.wikidata.org/' in u['expanded_url'] or 'w.wiki/' in u['expanded_url'] or 'tinyurl.com/' in u['expanded_url']:
            try:
                query_dict = {}
                query = parse_tweet_links(u['expanded_url'])
                if t['tweet']['full_text']:
                    text = t['tweet']['full_text']
                    query_dict['heading'] = text
                else:
                    continue
                query_dict['full_text'] = ''
                query_dict['text_query'] = ''

                # get all QIDs and PIDs
                qid_set = set(re.findall(r"Q\d+", f'{text}\n{query}'))
                pid_set = set(re.findall(r"P\d+", f'{text}\n{query}'))

                query_dict['qids'] = get_qid_titles(qid_set)
                query_dict['pids'] = get_pid_titles(pid_set)
                query_dict['sparql_query'] = query
                query_dict['title'] = '@WikidataFacts_Twitter_Archive'

                rows.append(query_dict)
            except:
                continue
    return rows

In [ ]:
tweets = []

with open('data/WikidataFacts-tweets.jsonl', 'r') as f:
    for l in f:
        t = json.loads(l)
        tweets += parse_tweet(t)

In [ ]:
pd.DataFrame(tweets).to_pickle('data/high-quality/tweets.pkl')

## Weekly query example pages
Sometimes Wikidata status updates (https://www.wikidata.org/wiki/Wikidata:Status_updates) include descriptive links to queries, which are aggregated in the Weekly query example archive (https://www.wikidata.org/wiki/Wikidata:Weekly_query_examples). This archive goes back almost 10 years.

In [ ]:
def parse_weekly_query_page(title):
    print(title)
    rows = []
    
    resp = session.get(
        formatversion=2,
        action='query',
        prop='revisions',
        rvslots='*',
        rvprop='content',
        titles=title
    )

    content = resp['query']['pages'][0]['revisions'][0]['slots']['main']['content']
    
    wikitext = parser.parse(content)

    for l in wikitext.filter_external_links():
        if 'query.wikidata.org/' in l or 'w.wiki/' in l:
            try:
                text, query = parse_external_wikilinks(l)
                query_dict = {}
                query_dict['heading'] = text
                query_dict['full_text'] = ''
                query_dict['text_query'] = ''

                # get all QIDs and PIDs
                qid_set = set(re.findall(r"Q\d+", f'{text}\n{query}'))
                pid_set = set(re.findall(r"P\d+", f'{text}\n{query}'))

                query_dict['qids'] = get_qid_titles(qid_set)
                query_dict['pids'] = get_pid_titles(pid_set)
                query_dict['sparql_query'] = query
                query_dict['title'] = title

                rows.append(query_dict)
            except:
                print('error')
                continue
    return rows

In [ ]:
session = mwapi.Session('https://wikidata.org', user_agent='mwapi sparql')
weekly_query = []

weekly_query_prefix = "Weekly query examples"
weekly_query_pages = get_pages_with_prefix(weekly_query_prefix, namespace=4)
for page in weekly_query_pages:
    weekly_query += parse_weekly_query_page(page['title'])

pd.DataFrame(weekly_query).to_pickle('data/high-quality/weekly-query.pkl')

## User pages with a suffix of "query"

In [ ]:
#tktk

## Aggregate all data sources

In [ ]:
qotw = pd.read_pickle('data/high-quality/qotw.pkl')
request = pd.read_pickle('data/high-quality/request-a-query.pkl')
example = pd.read_pickle('data/high-quality/example.pkl')
tfaq = pd.read_pickle('data/high-quality/tfaq.pkl')
weekly = pd.read_pickle('data/high-quality/weekly-query.pkl')
tweets = pd.read_pickle('data/high-quality/tweets.pkl')

In [ ]:
def format_df(df):
    df['heading'] = df['heading'].astype('string')
    return df

dfs = [qotw, request, example, tfaq, weekly, tweets]
dfs = [format_df(df) for df in dfs]

In [ ]:
all_pd = pd.concat(dfs).reset_index(drop=True)

In [ ]:
all_pd.to_pickle('data/high-quality/all.pkl')

### Drop non-english translations of queries

In [ ]:
suffixes = [
    'de', 'es', 'nl', 'es', 'fr', 'hy', 'ja', 'sv', 'tr', 'uk',
    'zh', 'ar', 'be-tarask', 'ca', 'cs', 'da', 'eo', 'eu', 'he',
    'arz', 'id', 'it', 'pl', 'sv', 'th', 'ko', 'lt', 'ms', 'pt-br',
    'ru', 'ro', 'si', 'vec'
]

suffixes = [f'/{s}' for s in translated_suffixes]

In [ ]:
all_no_translation = all_pd[
    ~all_pd.apply(
        lambda row: any(row.str.endswith(suffix).any() for suffix in suffixes), axis=1
    )
]

In [ ]:
all_no_translation

In [ ]:
all_no_translation.to_pickle('data/high-quality/all_no_translation.pkl')

In [2]:
import pandas as pd

In [2]:
df = pd.read_pickle('data/high-quality/all_no_translation.pkl')

In [3]:
df = df.reset_index(drop=True)

In [7]:
df.columns

Index(['heading', 'full_text', 'text_query', 'qids', 'pids', 'sparql_query',
       'title'],
      dtype='object')

In [23]:
df[df.text_query == '']['heading'][400]

'Women who have been elected to the National Academy of Sciences'

In [18]:
print(df['heading'][5000])

=== Lexemes by number of statements ===


In [26]:
import pickle5 as pickle

In [29]:
df = pd.read_pickle('data/high-quantity/wikidata-sparql-templates-bug-fixes.pkl')

ValueError: unsupported pickle protocol: 5